# Comprehensive Homework: Build and Test a Mini RAG System from Scratch 🧠

> **🎯 Today's Goal**: Combine the knowledge from the first three lessons (Embeddings, Retrieval, Generation) to build a functional Retrieval-Augmented Generation (RAG) system from scratch. Then, test it with a self-assessment!

In [ ]:
!pip install sentence-transformers transformers torch

## ⚙️ Part 1: The Retriever - Finding the Right Knowledge

First, we'll set up our Retriever. Its job is to take a question and find the most relevant piece of text from our knowledge base.

1.  **Load the Embedding Model** (`all-MiniLM-L6-v2`)
2.  **Create our Knowledge Base**
3.  **Encode Everything into Embeddings**
4.  **Calculate Similarity** to find the best match

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

print("✅ Libraries imported successfully!")

# 1. Load our embedding model
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Create a simple knowledge base
knowledge_base = [
    "The capital of France is Paris, a city famous for the Eiffel Tower and the Louvre museum.",
    "The Amazon rainforest is the world's largest tropical rainforest, known for its incredible biodiversity.",
    "Mount Everest is the highest mountain on Earth, located in the Himalayas.",
    "The Great Wall of China is a series of fortifications stretching over 13,000 miles.",
    "Photosynthesis is the process used by plants to convert light energy into chemical energy."
]

# 3. Encode our knowledge base into embeddings
knowledge_embeddings = retriever_model.encode(knowledge_base, convert_to_tensor=True)

print(f"✅ Retriever model loaded and knowledge base encoded with {len(knowledge_base)} documents.")

## ✍️ Part 2: The Generator - Extracting the Answer

Now we set up our Generator. This model will take the question and the context found by the retriever and extract the exact answer from it.

In [ ]:
# Load our question-answering (generator) model
generator = pipeline('question-answering', model='distilbert-base-cased-distilled-squad')

print("✅ Generator (QA) model loaded.")

## 🚀 Part 3: Testing our RAG System

Time to put it all together! The function below will simulate a full RAG pipeline and grade itself against a predefined set of questions and answers.

It will test two key things:
1.  **Retrieval Accuracy**: Did we find the right document?
2.  **Generation Accuracy**: Did we extract the correct answer from that document?

In [ ]:
def run_rag_assessment():
    """Runs a self-assessment of the RAG pipeline with multiple questions."""

    # Define our questions, expected context keywords, and expected answers
    test_questions = [
        {
            "question": "What is the highest mountain?",
            "expected_keyword": "Everest",
            "expected_answer": "Mount Everest"
        },
        {
            "question": "Which city is home to the Louvre museum?",
            "expected_keyword": "France",
            "expected_answer": "Paris"
        },
        {
            "question": "What process do plants use for energy?",
            "expected_keyword": "Photosynthesis",
            "expected_answer": "Photosynthesis"
        }
    ]

    score = 0
    total = len(test_questions) * 2 # 2 points per question (1 for retrieval, 1 for generation)

    print("--- 🚀 Starting RAG System Assessment ---\n")

    for i, test in enumerate(test_questions):
        question = test["question"]
        print(f"\n--- Question {i+1}: '{question}' ---")

        # --- 1. Retrieval Step ---
        question_embedding = retriever_model.encode(question, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(question_embedding, knowledge_embeddings)[0]
        top_result_index = torch.argmax(cos_scores)
        retrieved_context = knowledge_base[top_result_index]

        print(f"🔎  Retrieved Context: '{retrieved_context}'")

        # Check if the retrieval was correct
        if test["expected_keyword"] in retrieved_context:
            print("✅  Retrieval Correct!")
            score += 1
        else:
            print(f"❌  Retrieval Failed. Expected context with keyword: '{test['expected_keyword']}'")

        # --- 2. Generation Step ---
        qa_result = generator(question=question, context=retrieved_context)
        generated_answer = qa_result['answer']

        print(f"✍️  Generated Answer: '{generated_answer}'")

        # Check if the generation was correct
        if test["expected_answer"].lower() in generated_answer.lower():
            print("✅  Generation Correct!")
            score += 1
        else:
            print(f"❌  Generation Failed. Expected answer: '{test['expected_answer']}'")

    # --- Final Score ---
    print(f"\n--- 🏁 Assessment Complete ---")
    print(f"🎯 Final Score: {score} / {total}")
    if score == total:
        print("🎉🎉🎉 Perfect! Your RAG system is working as expected!")
    elif score >= total / 2:
        print("👍 Good job! The system is mostly correct.")
    else:
        print("🔧 The system ran into some issues. Review the steps and check the logic.")

# Run the assessment!
run_rag_assessment()

#  STUDENT TASKS 🧑‍💻

Now it's your turn to be the AI engineer. Your tasks are to run, analyze, and extend the RAG system you've just built.

### Task 1: Execute and Understand

Your first task is to simply run all the cells above and carefully read the output of the final self-assessment.

* **Observe the Score:** Did the system get a perfect score (6/6)?
* **Analyze Each Step:** For each question, look at the "Retrieved Context" and the "Generated Answer."
    * Did the retriever find the correct piece of knowledge?
    * Did the generator extract the right answer from that context?

### Task 2 (Challenge): Add a New Question

Your second task is to test the system with a new question about the **existing knowledge**.

**Instructions:**
1.  Copy the code from the cell below. It's the same assessment function as before, but with a new test question added.
2.  Run the cell and see if the system can answer correctly. The score should now be out of 8.

In [ ]:
# Task 2: Add a new question to the assessment function

def run_rag_assessment_task_2():
    test_questions = [
        {
        "question": "Which city is the capital of France?",
        "expected_keyword": "France",
        "expected_answer": "Paris"
        },
        {
        "question": "Where is the Amazon rainforest located?",
        "expected_keyword": "Amazon rainforest",
        "expected_answer": "world's largest tropical rainforest"
        },
        {
        "question": "What is the Great Wall of China known for?",
        "expected_keyword": "Great Wall of China",
        "expected_answer": "fortifications"
        },
        {
        "question": "How do plants produce energy from sunlight?",
        "expected_keyword": "Photosynthesis",
        "expected_answer": "Photosynthesis"
        }
    ]

    score = 0
    total = len(test_questions) * 2

    print("--- 🚀 Starting RAG System Assessment (Task 2) ---\n")

    for i, test in enumerate(test_questions):
        question = test["question"]
        print(f"\n--- Question {i+1}: '{question}' ---")
        question_embedding = retriever_model.encode(question, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(question_embedding, knowledge_embeddings)[0]
        top_result_index = torch.argmax(cos_scores)
        retrieved_context = knowledge_base[top_result_index]
        print(f"🔎  Retrieved Context: '{retrieved_context}'")
        if test["expected_keyword"] in retrieved_context:
            print("✅  Retrieval Correct!")
            score += 1
        else:
            print(f"❌  Retrieval Failed. Expected context with keyword: '{test['expected_keyword']}'")
        qa_result = generator(question=question, context=retrieved_context)
        generated_answer = qa_result['answer']
        print(f"✍️  Generated Answer: '{generated_answer}'")
        if test["expected_answer"].lower() in generated_answer.lower():
            print("✅  Generation Correct!")
            score += 1
        else:
            print(f"❌  Generation Failed. Expected answer: '{test['expected_answer']}'")

    print(f"\n--- 🏁 Assessment Complete ---")
    print(f"🎯 Final Score: {score} / {total}")
    if score == total:
        print("🎉🎉🎉 Perfect! Your RAG system handled the new question!")

# Run the updated assessment
run_rag_assessment_task_2()

### Task 3 (Advanced Challenge): Add New Knowledge & Test It

Your final and most important task is to **expand the RAG system's knowledge base** and then test it.

**Instructions:**
1.  **Add a new fact** to the `knowledge_base` in the code cell below.
2.  **You must re-run this cell** to update the `knowledge_embeddings`! The system won't know about the new fact until you do.
3.  Finally, run the last code cell, which has a new test question about the knowledge you just added.

In [ ]:
# Task 3, Step 1: Add a new sentence to the knowledge base

knowledge_base_task_3 = [
    "The Moon is the Earth's only natural satellite and affects tides."
    "The fastest land animal is the cheetah, capable of speeds up to 75 mph."
    "Mercury is the closest planet to the Sun and has no substantial atmosphere."

]

# Re-encode the updated knowledge base
knowledge_embeddings_task_3 = retriever_model.encode(knowledge_base_task_3, convert_to_tensor=True)

print(f"✅ Knowledge base updated and re-encoded with {len(knowledge_base_task_3)} documents.")

In [ ]:
# Task 3, Step 2: Test your newly added knowledge

def run_rag_assessment_task_3():
    test_questions = [
        # --- NEW QUESTION FOR YOUR NEW KNOWLEDGE ---
        {
            "question": "What is Earth's only natural satellite?",
            "expected_keyword": "Moon",
            "expected_answer": "Moon"
        }
    ]

    score = 0
    total = len(test_questions) * 2

    print("--- 🚀 Starting RAG System Assessment (Task 3) ---\n")

    for i, test in enumerate(test_questions):
        question = test["question"]
        print(f"\n--- Question {i+1}: '{question}' ---")
        # Use the updated embeddings from Task 3
        question_embedding = retriever_model.encode(question, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(question_embedding, knowledge_embeddings_task_3)[0]
        # Use the updated knowledge base from Task 3
        top_result_index = torch.argmax(cos_scores)
        retrieved_context = knowledge_base_task_3[top_result_index]
        print(f"🔎  Retrieved Context: '{retrieved_context}'")
        if test["expected_keyword"] in retrieved_context:
            print("✅  Retrieval Correct!")
            score += 1
        else:
            print(f"❌  Retrieval Failed. Expected context with keyword: '{test['expected_keyword']}'")
        qa_result = generator(question=question, context=retrieved_context)
        generated_answer = qa_result['answer']
        print(f"✍️  Generated Answer: '{generated_answer}'")
        if test["expected_answer"].lower() in generated_answer.lower():
            print("✅  Generation Correct!")
            score += 1
        else:
            print(f"❌  Generation Failed. Expected answer: '{test['expected_answer']}'")

    print(f"\n--- 🏁 Assessment Complete ---")
    print(f"🎯 Final Score: {score} / {total}")
    if score == total:
        print("🏆🏆🏆 Success! You have successfully extended the knowledge of your RAG system!")

# Run the final assessment
run_rag_assessment_task_3()

In [ ]:
# Install Streamlit and pyngrok
!pip install streamlit -q
!pip install pyngrok -q


In [ ]:
%%writefile app.py
import streamlit as st
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import torch

# ------------------------------
# Load models
# ------------------------------
@st.cache_resource
def load_models():
    retriever = SentenceTransformer("all-MiniLM-L6-v2")
    generator = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
    return retriever, generator

retriever_model, generator = load_models()

# Knowledge base
knowledge_base = [
    "The capital of France is Paris, a city famous for the Eiffel Tower and the Louvre museum.",
    "The Amazon rainforest is the world's largest tropical rainforest, known for its incredible biodiversity.",
    "Mount Everest is the highest mountain on Earth, located in the Himalayas.",
    "The Great Wall of China is a series of fortifications stretching over 13,000 miles.",
    "Photosynthesis is the process used by plants to convert light energy into chemical energy."
]
knowledge_embeddings = retriever_model.encode(knowledge_base, convert_to_tensor=True)

# ------------------------------
# Streamlit UI
# ------------------------------
st.title("🧠 Mini RAG System Demo")
st.write("Ask a question and the system will find the most relevant context and generate an answer!")

question = st.text_input("Enter your question:")

if st.button("Get Answer") and question:
    question_embedding = retriever_model.encode(question, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(question_embedding, knowledge_embeddings)[0]
    top_result_index = torch.argmax(cos_scores)
    retrieved_context = knowledge_base[top_result_index]

    st.subheader("🔍 Retrieved Context")
    st.write(retrieved_context)

    qa_result = generator(question=question, context=retrieved_context)
    st.subheader("✍️ Generated Answer")
    st.success(qa_result['answer'])
else:
    st.info("Enter a question and click 'Get Answer'.")


In [ ]:
from pyngrok import ngrok
import time
import os

# ضع توكن ngrok هنا (استبدلي بالقيمة الجديدة بعد ما تلغي القديمة)
ngrok.set_auth_token("33NNp4yunF3sa59CtjWFeS7oiO0_2Fk22thZL69xzVupzp1YZ")

# افتح نفق على البورت 8501
public_url = ngrok.connect(8501)
print("Ngrok tunnel:", public_url.public_url)

# شغّل streamlit في الخلفية
# المخرجات تُوجّه إلى /dev/null لتفادي فوضى logs في الخلية
os.system("streamlit run app.py --server.port 8501 &>/dev/null &")

# انتظر شوية لين يبدا السيرفر
time.sleep(5)
print("Open the public URL above in your browser.")
